<a href="https://colab.research.google.com/github/dotdak/Hackathon/blob/master/ZaloAIHackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')
!ls -la "drive/My Drive/darknet"

In [0]:
!curl --output images.zip https://dl.challenge.zalo.ai/ZaloAIHackathon2019/zai2019_hackathon_train.zip
!unzip images.zip

In [0]:
# We're unzipping the cuDNN files from your Drive folder directly to the VM CUDA folders
!tar -xzvf drive/My\ Drive/darknet/cuDNN/cudnn-10.0-linux-x64-v7.5.0.56.tgz -C /usr/local/
!chmod a+r /usr/local/cuda/include/cudnn.h

# Now we check the version we already installed. Can comment this line on future runs
!cat /usr/local/cuda/include/cudnn.h | grep CUDNN_MAJOR -A 2

In [0]:
!git clone https://github.com/kriyeng/darknet/
%cd darknet
!git checkout feature/google-colab
!make
!cp -r . ../drive/My\ Drive/darknet/bin
!/usr/local/cuda/bin/nvcc --version
%cd ..

In [0]:
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  #plt.rcParams['figure.figsize'] = [10, 5]
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()
  
  
def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)
def download(path):
  from google.colab import files
  files.download(path)

In [0]:
import cv2
import json
import os
import numpy as np
import pandas as pd
BASE_PATH = '/content/drive/My Drive/'
IMAGES_PATH = BASE_PATH + 'zai2019_hackaton_train/images/train/'
TEST_PATH = BASE_PATH + 'zai2019_hackaton_train/images'
ANNOTATION = BASE_PATH + 'zai2019_hackaton_train/annotations/ninedash_keypoints_train.json'
CROPPED_PATH = BASE_PATH + 'zai2019_hackaton_train/cropped_images/'
DARKNET = BASE_PATH + 'darknet/bin/darknet'

In [0]:
with open(ANNOTATION) as f:
  config = json.load(f)

In [0]:
images_name_list = [f for r, d, f in os.walk(IMAGES_PATH)][0]
id_name_mapping = {int(f.split('.')[0]): f for f in images_name_list}

In [0]:
# def get_image_name(image_id):
#   return id_name_mapping[image_id]

# def save_cropped_image(image_name, param):
#   image = cv2.imread(IMAGES_PATH + image_name)
#   [a, b, c, d] = [val if val >= 0 else 0 for val in param]
#   [name, extension] = image_name.split('.')
#   count = 1
#   file_name = image_name
#   while os.path.isfile(CROPPED_PATH + file_name):
#     suffix = '_{}'.format(count)
#     file_name = name + suffix + '.' + extension
#     count += 1
#   cv2.imwrite(CROPPED_PATH + file_name, image[b:b+d, a:a+c])
# def test_cropped(image_id, param):
#   save_cropped_image(get_image_name(image_id), param)

# for annotation in config['annotations']:
#   image_name = get_image_name(annotation['image_id'])
#   param = annotation['bbox']
#   if annotation['area'] > 0:
#     save_cropped_image(image_name, param)

In [0]:
# test_cropped(5, [569, 0, 531, 918])

In [0]:
# annotations = pd.read_csv('annotations.csv', index_col = 'id')
# images = pd.read_csv('images.csv', index_col = 'id')

In [0]:
YOLOBOX = BASE_PATH + 'yolobox/'
for i in images.index:
  name = images['file_name'][i]
  annotation = annotations[annotations['image_id'] == i]
  height = int(images['height'][i])
  width = int(images['width'][i])
  with open(YOLOBOX + name[:-4] + '.txt', 'w') as f:
    for j in range(len(annotation)):
      bbox = annotation.iloc[j][1]
      bbox = bbox[1:-1]
      bbox = bbox.split(',')
      bbox = [int(i) if int(i) >= 0 else 0 for i in bbox]
      f.write(f'0 {(bbox[0] + bbox[2]/2)/width} {(bbox[1] + bbox[3]/2)/height} {bbox[2]/width} {bbox[3]/height}')
      if j != len(annotation) - 1:
        f.write('\n')
!cp /content/drive/My\ Drive/yolobox/* /content/drive/My\ Drive/zai2019_hackaton_train/images/train/

In [0]:
list_files = [f for r, d, f in os.walk(IMAGES_PATH)][0]
list_dir = [IMAGES_PATH + f for f in list_files]
with open(DARKNET + 'train.txt', 'w') as f:
  for line in list_dir:
    f.write(line + '\n')


In [0]:
# Training phase
!/content/darknet/darknet detector train "/content/drive/My Drive/darknet/obj.data" "/content/drive/My Drive/darknet/yolov3.cfg" "/content/drive/My Drive/darknet/bin/darknet/weights/darknet53.con.74.weights" -dont_show

In [43]:
# Remove previous detection results
%rm /content/drive/My\ Drive/results/*

done


In [0]:
# Detecting phase
!python /content/drive/My\ Drive/darknet/YOLO.py -i "/content/image_contrast/" -o "/content/drive/My Drive/results/" -cl /content/drive/My\ Drive/darknet/yolov3.txt -w /content/drive/My\ Drive/darknet/backup2/yolov3_last.weights -c /content/drive/My\ Drive/darknet/yolov3.cfg
%cd /content/drive/My\ Drive/darknet
!python parse_zai2019_prediction.py

In [0]:
# %cd /content/
# !gdown https://drive.google.com/uc?id=1AzWtB457hvjUj9YdQVAr9aM-3GMYN9X4
# !unzip /content/contrast.zip